In [ ]:
# Import necessary libraries
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import json
import plotly.graph_objects as go

In [ ]:
url  = [ #
    #ibra50
    'https://raw.githubusercontent.com/BDonadelli/Finance-playground/main/data/Cart_IBr50.csv',
    #ibra100
    'https://raw.githubusercontent.com/BDonadelli/Finance-playground/main/data/Cart_IBr100.csv',
    #div
    'https://raw.githubusercontent.com/BDonadelli/Finance-playground/main/data/Cart_Idiv.csv' ,
    #small
    'https://raw.githubusercontent.com/BDonadelli/Finance-playground/main/data/Cart_Small.csv',
    #ibbr
    'https://raw.githubusercontent.com/BDonadelli/Finance-playground/main/data/Cart_IBBR.csv',
    #low vol
    'https://raw.githubusercontent.com/BDonadelli/Finance-playground/main/data/Cart_Ibov_LowVol.csv'
]

escolha = 0 #int(input('0-ibra50; 1-ibra100; 2-idiv; 3-small; 4- ibbr ; 5-low vol: '))
indice = { 0:'Ibra50' , 1:'Ibra100' , 2:'Idiv' , 3:'Small' , 4:'IBBr' , 5:'Low Vol'}

In [ ]:
import os
from urllib.parse import urlparse

colunas = [ 'Codigo', 'Acao', 'Tipo', 'Qtde_Teorica', 'Participacao']

arquivo = os.path.basename(urlparse(url[escolha]).path) 
# Caminho do arquivo local
file_path = os.path.expanduser(f'~/GHub/Finance-playground/data/{arquivo}')

# Verificar se o arquivo existe localmente
if os.path.exists(file_path):
    # Ler o arquivo local
    ticker_list = pd.read_csv( file_path , sep=';' , decimal=',' , thousands='.' , 
                header=None , skiprows=2 , skipfooter=2 , names=colunas ,
                encoding='latin1',index_col=False , engine='python')
    ticker_list=ticker_list['Codigo'].to_list()
    print("Arquivo lido localmente.")
else:
    # Ler o arquivo a partir da URL
    ticker_list = pd.read_csv( url[escolha] , sep=';' , decimal=',' , thousands='.' , 
                header=None , skiprows=2 , skipfooter=2 , names=colunas ,
                encoding='latin1',index_col=False , engine='python')
    ticker_list=ticker_list['Codigo'].to_list()
    print("Arquivo lido da URL.")

In [ ]:
years=10
end_date = datetime.now()
start_date = end_date - timedelta(days=years * 365)
stock_data = pd.DataFrame()


frames = []

for ticker in ticker_list:
    stock = yf.Ticker(ticker+'.SA')
    hist_data = stock.history( start=start_date, end=end_date)
    if hist_data.empty or 'Close' not in hist_data:
        print(f"Pulando {ticker}")
        continue
    frames.append(hist_data['Close'].rename(ticker))

stock_data = pd.concat(frames, axis=1)
    
    # close_data = hist_data['Close'].rename(ticker)
    # stock_data = pd.merge(stock_data, pd.DataFrame(close_data), left_index=True, right_index=True, how='outer')

In [ ]:
daily_data = stock_data.copy()
daily_data

In [ ]:
def resample_data(data, period):
    if period == 'D':
        return data
    elif period == 'W':
        return data.resample('W').last()
    elif period == 'M':
        return data.resample('ME').last()


def momentum_strategy(data, initial_amount, top_n, tax_rate = 0.15 , period='M'):
    data = resample_data(data, period)
    log_returns = np.log(data / data.shift(1))
    simulation_details = pd.DataFrame(index=log_returns.index,
                                      columns=['Portfolio', 'Profit Before Tax', 'Tax', 'Portfolio Value'])
    cash = initial_amount

    # strategy
    for i in range(0, len(log_returns) - 1):
        # top n momentum 
        top_stocks = log_returns.iloc[i].sort_values(ascending=False).head(top_n)
        top_stocks = top_stocks[top_stocks > 0]

        if not top_stocks.empty:
            simulation_details.loc[log_returns.index[i + 1], 'Portfolio'] = json.dumps(top_stocks.index.tolist())
            num_stocks = len(top_stocks)
            allocation_per_stock = cash / num_stocks
            # Calculo do valor do novo portfólio com base nos retornos do dia seguinte
            new_value = sum(allocation_per_stock * np.exp(log_returns.loc[log_returns.index[i + 1], stock]) for stock in top_stocks.index)
            #  Calcular e deduzir o imposto se houver lucro
            profit = new_value - cash
            simulation_details.loc[log_returns.index[i + 1], 'Profit Before Tax'] = round(profit, 2)

            if profit > 0:
                tax = profit * tax_rate
                new_value -= tax
                simulation_details.loc[log_returns.index[i + 1], 'Tax'] = round(tax, 2)
            simulation_details.loc[log_returns.index[i + 1], 'Portfolio Value'] = round(new_value, 2)

        else:
            ## Sem alocação, então o valor do portfólio permanece o mesmo
            simulation_details.loc[log_returns.index[i + 1], 'Portfolio Value'] = cash
        # Atualizar valor em dinheiro para a próxima rodada
        cash = simulation_details.loc[log_returns.index[i + 1], 'Portfolio Value']
    # Atribuir o valor inicial à primeira linha
    simulation_details.loc[log_returns.index[0], 'Portfolio Value'] = initial_amount
    return simulation_details

In [ ]:
initial_amount = 10000
top_n = 5
frequency='M'
tax_rate=.15
rf = 0.11
bench_annual_rate = 0.06

simulation_details = momentum_strategy(daily_data, initial_amount, top_n,tax_rate,frequency)
simulation_details

In [ ]:
# Simula o desempenho de cada ação individual no mesmo período
def track_individual_investments(data, initial_amount, simulation_details, period='W'):
    # Reamostrar dados com base no período especificado e calcular retornos 
    data = resample_data(data, period)
    returns = data.pct_change()
    # armazenar valores de ações individuais ao longo do tempo
    individual = pd.DataFrame(index=data.index, columns=data.columns)
    for stock in data.columns:
        # Simular um investimento em cada ação
        individual[stock] = (1 + returns[stock]).cumprod() * initial_amount
    # Incluir o Valor do Portfólio da estratégia de momentum
    individual['Portfolio Value'] = simulation_details['Portfolio Value']
    individual['Baseline'] = individual.iloc[:, :-1].T.mean()
    # Ajustar os primeiros valores para corresponder ao Valor Inicial.
    individual.iloc[0, :] = initial_amount
    return individual.fillna(0).astype(int)

individual_investments_df = track_individual_investments(daily_data, initial_amount, simulation_details, frequency)

individual_investments_df

In [ ]:
from scipy.stats import ttest_1samp

def sharpe_ratio(returns, annual_risk_free_rate=0.11, frequency='D'):
    if frequency == 'D':
        adjusted_rfr = (1 + annual_risk_free_rate) ** (1/252) - 1
    elif frequency == 'W':
        adjusted_rfr = (1 + annual_risk_free_rate) ** (1/52) - 1
    elif frequency == 'M':
        adjusted_rfr = (1 + annual_risk_free_rate) ** (1/12) - 1

    return (returns - adjusted_rfr).mean() / (returns - adjusted_rfr).std()

def t_test_portfolio_returns(portfolio_returns, bench_annual_rate=0.1, frequency='D'):
    '''
    realiza um teste t de Student de uma amostra para verificar se os retornos médios de portfólio 
    são significativamente diferentes de uma taxa de referência anual ajustada

    Args:
    portfolio_returns:  Série de retornos do portfólio (diários, semanais, mensais, etc.) do portfólio.
    bench_annual_rate:  A taxa de referência anual contra a qual se deseja comparar os retornos do portfólio. 
                        Por padrão, é 0.1 (ou seja, 10% ao ano).
    frequency: Frequência dos retornos fornecidos em portfolio_returns. 

    Returns: 
    Retorna uma tupla (t_stat, p_value) contendo a estatística t e o valor p do teste. Se o valor p for pequeno, 
    rejeita-se a hipótese nula de que os retornos do portfólio não diferem significativamente da taxa de referência ajustada.
    '''


    if frequency == 'D':
        adjusted_rfr = (1 + bench_annual_rate) ** (1/252) - 1
    elif frequency == 'W':
        adjusted_rfr = (1 + bench_annual_rate) ** (1/52) - 1
    elif frequency == 'M':
        adjusted_rfr = (1 + bench_annual_rate) ** (1/12) - 1

    # compara a média dos retornos do portfólio com a adjusted_rfr para verificar se a diferença entre eles é estatisticamente significativa.
    t_stat, p_value = ttest_1samp(portfolio_returns[1:], adjusted_rfr) 
    return t_stat, p_value

def performance_metrics(dataframe, initial_amount, bench_annual_rate, frequency='D'):
    final_values = dataframe.iloc[-1]
    relative_values = final_values / initial_amount - 1 

    returns = dataframe.pct_change()

    if frequency == 'D':
        annualization_factor = 252
    elif frequency == 'W':
        annualization_factor = 52
    elif frequency == 'M':
        annualization_factor = 12

    # Anualização
    mean_returns = (1 + returns.mean()) ** annualization_factor - 1
    sharpes = returns.apply(sharpe_ratio, annual_risk_free_rate= rf, frequency=frequency)

    # Testa se os retornos da carteira são consistentes
    portfolio_returns = dataframe['Portfolio Value'].pct_change()
    t_stat, p_value = t_test_portfolio_returns(portfolio_returns, bench_annual_rate, frequency=frequency)

    return final_values, relative_values, mean_returns, sharpes, t_stat, p_value / 2



In [ ]:
final_values, relative_values, mean_returns, sharpes, t_stat, p_value = performance_metrics(individual_investments_df, initial_amount, bench_annual_rate, frequency)

In [ ]:
bench = yf.download("BOVA11.SA", start=start_date , end=end_date , auto_adjust=True , progress=False)['Close']
bench = (bench/bench.iloc[0])*initial_amount
bench= resample_data(bench,frequency)
bench

In [ ]:
bench.pct_change().dropna().mean()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = go.Figure()
# Portfolio Value
fig.add_trace(go.Scatter(x=individual_investments_df.index,
                             y=individual_investments_df['Portfolio Value'],
                             mode='lines',
                             name='Portfolio Value',
                             line=dict(color='blue', width=2.5)),
                  )
    # carteira do indice
fig.add_trace(go.Scatter(x=individual_investments_df.index,
                             y=bench,
                             mode='lines',
                             name='Benchmark Value',
                             line=dict(color='red', width=2.5)),
                  )


# T-test and P-value
significance_text = f"<b>T-test:</b> {t_stat:.2f}<br><b>P-value:</b> {p_value:.5f}"
if t_stat > 2 and p_value < 0.05:
    significance_text += f"<br><b>Significamente diferente de {bench_annual_rate:.0%} a.a.</b>"


fig.update_layout(title_text="valor da carteira com a estratégia e valor da carteira do ETF BOVA11",
                      title_font=dict(size=18, color='black', family="Arial Black"),
                      title_pad=dict(t=10),
                      title_x=0.5,
                      bargap=0.05,
                      annotations=[
        dict(
            text=significance_text, 
            showarrow=False, 
            xref="paper", 
            yref="paper", 
            x=.5,  
            y=-0.25,  
            font=dict(size=12, color="black"),  # Estilo da fonte
            align="center"  # Alinhamento do texto
        )
    ]
                      )

fig.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_combined_charts(dataframe, final_values, relative_values, sharpes, mean_returns):
    labels = final_values.index
    colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A']

    fig = make_subplots(rows=2, cols=2,
                        subplot_titles=('Valor final dos ativos',
                                        'Taxa de crescimento no período',
                                        'Sharpe anaulizado',
                                        'Retorno anualizado'),
                        vertical_spacing=0.1)

    

    # Final values
    fig.add_trace(go.Bar(x=labels,
                         y=final_values.values,
                         name='Final Values ($)',
                         text=[f"${v:,.2f}" for v in final_values.values],
                         textposition='outside',
                         marker_color=colors[1]),
                  row=1, col=1)

    # Relative Growth
    fig.add_trace(go.Bar(x=labels,
                         y=relative_values.values,
                         name='Relative Growth',
                         text=[f"{v:.2%}" for v in relative_values.values],
                         textposition='outside',
                         marker_color=colors[2]),
                  row=1, col=2)

    # Sharpe Ratios
    fig.add_trace(go.Bar(x=labels,
                         y=sharpes.values,
                         name='Annualized Sharpe Ratio',
                         text=[f"{v:.2f}" for v in sharpes.values],
                         textposition='outside',
                         marker_color=colors[3]),
                  row=2, col=1)

    # Mean Returns
    fig.add_trace(go.Bar(x=labels,
                         y=mean_returns.values,
                         name='Annualized Mean Returns',
                         text=[f"{v:.2%}" for v in mean_returns.values],
                         textposition='outside',
                         marker_color=colors[4]),
                  row=2, col=2)

    # Update layout
    fig.update_layout(title_text="Métricas individuais",
                      title_font=dict(size=24, color='black', family="Arial Black"),
                      title_pad=dict(t=10),
                      showlegend=False,
                      height=1000,
                      title_x=0.5,
                      bargap=0.05,
                      )

    fig.show()

plot_combined_charts(individual_investments_df, final_values, relative_values, sharpes, mean_returns)